In [1]:
class Account:
    def __init__(self, account_number, account_holder,password ,  balance=0):
      '''intial constructor
        initializes the object
      '''
      self.account_number = account_number
      self.account_holder = account_holder
      self.balance = balance
      self.transaction_history = []
      self.password = password

#####################################################################################

    def deposit(self, amount):
      """
      description of this function
      add the amount to the balance
      Ex: if the balance is 100 and you deposit 50, the balance should be 150
      parameter: amount
      type: int
      """
      #check if amount < = 0 because the amount must be positive
      if amount <= 0:
          print("Deposit amount must be positive.")
      else:      # if amount is positive 
        self.balance += amount          #self.balance = self.balance + amount
        self.transaction_history.append(f"Deposited: ${amount}")
        print(f"Deposited ${amount}. New balance: ${self.balance}")

######################################################################################

    def withdraw(self, amount):
        """
        description of this function
        subtract the amount from the balance
        Ex: if the balance is 100 and you deposit 50, the balance should be 50
        parameter: amount
        type: int
        """
        #check if amount < = 0 because the amount must be positive
        if amount <= 0:
            print("Withdrawal amount must be positive.")
            
        elif amount > self.balance:   # amount must be greater than current balance
            print("Insufficient funds.")
            
        else:
            self.balance -= amount          # self.balance=self.balance-amount
            self.transaction_history.append(f"Withdrew:{amount}")
            print(f"Withdrew {amount}. New balance:{self.balance}")

#####################################################################################

    def check_balance(self):
      """
      description of this function
      print the balance
      """
      print(f'Current balance:{self.balance}')

#####################################################################################

    def print_transaction_history(self):
        """
        describtion of this function :
        print Transaction history for account  
        """
      
        print(f"Transaction history for account {self.account_number}:")
        for transaction in self.transaction_history:
            print(transaction)

#####################################################################################

    def change_password(self):
        """
        describtion of this function : if the user need to change the password of your account
        #No parameter
        
        """     
        current_password = input("Enter current password: ")
        #require from user to enter the old password
        if current_password == self.password:
            ##require from user to enter the new password
            new_password = input("Enter new password: ")
            #to confirm password
            confirm_password = input("Confirm new password: ")
            if new_password == confirm_password:
                self.password = new_password
                print("Password changed successfully.")
            else:
                print("Passwords do not match.")
        else:
            print("Current password is incorrect.")
            
#####################################################################################

    def to_dict(self):
        return {
            "account_holder": self.account_holder,
            "balance": self.balance,
            "password": self.password,
            "transaction_history": self.transaction_history
        }


In [2]:
#####################################################################################
import json
import os
#####################################################################################

class ReadAccount:
    def __init__(self):
        self.accounts = {}
        self.load_accounts_from_file()

    def get_file_path(self):
        """Returns the file path for accounts.json"""
        return os.path.join(os.getcwd(), 'accounts.json')

    def load_accounts_from_file(self):
        """Loads account data from a JSON file"""
        file_path = self.get_file_path()
        if os.path.exists(file_path):
            try:
                with open(file_path, 'r') as file:
                    file_content = file.read()
                    if file_content:
                        accounts_data = json.loads(file_content)
                        for account_number, details in accounts_data.items():
                            account = Account(
                                account_number,
                                details['account_holder'],
                                details['password'],
                                details['balance']
                            )
                            account.transaction_history = details['transaction_history']
                            self.accounts[account_number] = account
                    else:
                        print("Accounts file is empty. Starting with no accounts.")
            except json.JSONDecodeError:
                print("Error reading the accounts file. Starting fresh.")
        else:
            print("Accounts file not found. Creating a new one.")
            with open(file_path, 'w') as file:
                json.dump({}, file)

    def save_accounts_to_file(self):
        """Saves account data to a JSON file"""
        file_path = self.get_file_path()
        accounts_to_save = {account_number: account.to_dict() for account_number, account in self.accounts.items()}
        with open(file_path, 'w') as file:
            json.dump(accounts_to_save, file, indent=4)

In [3]:
import random
import datetime
import string

class BankingSystem:
    def __init__(self):
        """
        Initial constructor
        Initializes the object
        """
        # Take the instance from the class (ReadAccount())
        self.read_account = ReadAccount()
        # Load accounts from the file
        self.accounts = self.read_account.accounts

        # Add a dictionary to store last login times
        self.last_logins = {}

#####################################################################################

    def create_account(self):
        """
        Description of function: to create a new account
        Requires 3 inputs from the user:
        first input: input account number
        type:int
        second input: input account_holder
        type : string or int
        third input: input password
        type : string or int
        """
        account_number = input("Enter account number: ")
        account_holder = input("Enter account holder name: ")
        password = input("Set a password for the account: ")

        if account_number in self.accounts:
            print("Account already exists.")

        else:
            balance_input = input("Enter initial balance (default is 0): ")
            balance = float(balance_input) if balance_input else 0

            # Take the instance from the class Account
            new_account = Account(account_number, account_holder, password, balance)
            self.accounts[account_number] = new_account
            print(f"Account created successfully for {account_holder}.")

            # Initialize last login time for the new account
            self.last_logins[account_number] = datetime.datetime.now()
#####################################################################################
    def forgot_password(self, account_number):
            account = self.accounts.get(account_number)
            if account:
                # Generate a secure, random password
                new_password = ''.join(random.SystemRandom().choices(
                    string.ascii_letters + string.digits + string.punctuation, k=12))

                # Send password reset instructions to the account holder's email (if available)
                # (This functionality is not implemented in this example due to external dependencies)
                # Placeholder:
                print(f"A temporary password has been generated and sent to the email associated with account {account_number}.")
                print("Please update your password upon login.")

                # Update the password in memory and on disk securely
                account.password = new_password
                self.read_account.save_accounts_to_file()

                # Additional security measures (consider implementing):
                # - Implement account lockout after a certain number of failed login attempts.
                # - Offer two-factor authentication for additional security.

            else:
                print("Account not found.")



#####################################################################################

    def login(self):
        account_number = input("Enter account number: ")
        password = input("Enter password: ")
        account = self.accounts.get(account_number)

        if account:
            incorrect_password_attempts = 0
            while incorrect_password_attempts < 2:
                if account.password == password:
                    key_verify = random.randrange(10000, 1000000)
                    print(f"The verification code: {key_verify}")

                    verify_input = int(input("Please enter the verification code: "))
                    incorrect_attempts = 0
                    while incorrect_attempts < 3:
                        if verify_input == key_verify:
                            print(f"Welcome, {account.account_holder}!")
                            self.last_logins[account_number] = datetime.datetime.now()
                            return account
                        else:
                            incorrect_attempts += 1
                            print("Verification code is not correct. Please try again.")
                            verify_input = int(input("Please enter the verification code: "))

                    if incorrect_attempts == 3:
                        print("Too many incorrect attempts. Would you like me to resend the verification code?")
                        resend_choice = input("Enter 'yes' or 'no': ")
                        if resend_choice.lower() == 'yes':
                            print(f"Resending verification code: {key_verify}")
                        else:
                            print("Login failed.")
                    break
                else:
                    incorrect_password_attempts += 1
                    print("Incorrect password. Please try again.")
                    if incorrect_password_attempts == 2:
                        print("Did you forget your password? Enter 'yes' or 'no':")
                        forgot_pass = input()
                        if forgot_pass.lower() == 'yes':
                            self.forgot_password(account_number)
                        else:
                            print("Login failed.")
        else:
            print("Account not found.")

        return None

########################################################################################

    def transfer_funds(self, account_from, account_to_number, amount):
        if account_to_number in self.accounts:
            account_to = self.accounts[account_to_number]
            if account_from.balance >= amount:
                account_from.withdraw(amount)
                account_to.deposit(amount)
                print(f"Transferred {amount} from {account_from.account_holder} to {account_to.account_holder}.")
                self.read_account.save_accounts_to_file()
            else:
                print("Insufficient funds for the transfer.")
        else:
            print("Destination account not found.")

In [4]:
if __name__ == "__main__":
    banking_system = BankingSystem()

    while True:
        print("1. Create Account")
        print("2. Login")
        print("3. Exit")
        choice = input("Choose an option: ")

        if choice == "1":
            banking_system.create_account()
        elif choice == "2":
            account = banking_system.login()
            if account:
                while True:
                    print("\n1. Deposit")
                    print("2. Withdraw")
                    print("3. Check Balance")
                    print("4. Transaction History")
                    print("5. Change Password")
                    print("6. Transfer Funds")
                    print("7. Logout")
                    action = input("Choose an action: ")

                    if action == "1":
                        try:
                            amount = float(input("Enter deposit amount: "))
                            account.deposit(amount)
                        except ValueError:
                            print("Invalid amount. Please enter a numeric value.")
                    elif action == "2":
                        try:
                            amount = float(input("Enter withdrawal amount: "))
                            account.withdraw(amount)
                        except ValueError:
                            print("Invalid amount. Please enter a numeric value.")
                    elif action == "3":
                        account.check_balance()
                    elif action == "4":
                        account.print_transaction_history()
                    elif action == "5":
                        account.change_password()
                    elif action == "6":
                        account_to_number = input("Enter destination account number: ")
                        try:
                            amount = float(input("Enter transfer amount: "))
                            banking_system.transfer_funds(account, account_to_number, amount)
                        except ValueError:
                            print("Invalid amount. Please enter a numeric value.")
                    elif action == "7":
                        print("Logging out.")
                        last_login = banking_system.last_logins.get(account.account_number)
                        if last_login:
                            print(f"Last login time: {last_login}")
                        break
                    else:
                        print("Invalid action.")
        elif choice == "3":
            print("Exiting the system. We wish you all the best.")
            break
        else:
            print("Invalid choice, please enter only 1, 2, or 3.")


1. Create Account
2. Login
3. Exit
